## Descobrindo padrões de perda de clientes

Analisarei os dados clientes perdidos para descobrir padrões. Isso ajudará a entender melhor estes padrões e fazer uma análise da causa raiz pela qual eles estão ocorrendo. Usarei o algoritmo de mineração de regras de associação para esse propósito.

## Carregando o conjunto de dados e transformando

In [ ]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt


In [ ]:
!pip install apyori

from apyori import apriori

In [ ]:
#Carregando o conjunto de dados dos clientes em potencial
raw_data = pd.read_csv("06_05_attrition.csv")

raw_data.head()

,LIFETIME,TYPE,REASON,AGE_GROUP,EMP_STATUS,MARITAL_STATUS,RENEWALS,PROBLEMS,OFFERS
0,1 - 3 M,CANCEL,BETTER DEALS,< 20,STUDENT,SINGLE,0,0 to 5,0 to 2
1,1 - 3 M,CANCEL,BETTER DEALS,< 20,STUDENT,SINGLE,0,0 to 5,0 to 2
2,1Y - 2Y,CANCEL,NOT HAPPY,30 - 50,EMPLOYED,MARRIED,1,10 plus,0 to 2
3,1Y - 2Y,EXPIRY,BETTER DEALS,30 - 50,EMPLOYED,MARRIED,1,0 to 5,2 to 5
4,1Y - 2Y,CANCEL,NOT HAPPY,30 - 50,UNEMPLOYED,SINGLE,1,10 plus,0 to 2


O CSV contém informações sobre cada cliente que deixou a empresa. Contém atributos como CICLO DE VIDA do cliente, como o cliente saiu, razões, problemas e dados demográficos.

Para fazer a mineração de regras de associação os dados precisam estar em um formato específico. Cada linha deve ser uma transação com uma lista de itens para essa transação. Converteremos os dados do CSV em valores como "name=value" para criar essa estrutura de dados específica.

In [ ]:
basket_str = ""
for rowNum, row in raw_data.iterrows():

    #Quebra de linhas
    if (rowNum != 0):
        basket_str = basket_str + "\n"
    #Adicionar rowid como primeira coluna
    basket_str = basket_str + str(rowNum)
    #Adicionar colunas
    for colName, col in row.iteritems():
        basket_str = basket_str + ",\"" + colName + "=" + str(col) +"\""

#print(basket_str)
basket_file=open("warranty_basket.csv","w")
basket_file.write(basket_str)
basket_file.close()


<ipython-input-10-d5845c3e8336>:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for colName, col in row.iteritems():


## Criando regras de associação
Usando o algoritmo apriori para criar regras de associação. A seguir, extraímos os resultados e preenchemos um data frame para uso futuro. O apriori proporciona o LHS para múltiplas combinações dos itens. Capturarei métricas “count”, “confidence” e “fit”

In [ ]:
basket_data=pd.read_csv("warranty_basket.csv",header=None)
filt_data = basket_data.drop(basket_data.columns[[0]], axis=1)
results= list(apriori(filt_data.values))

rulesList= pd.DataFrame(columns=('LHS', 'RHS', 'COUNT', 'CONFIDENCE','LIFT'))
rowCount=0

#converter resultados em um data frame
for row in results:
    for affinity in row[2]:
        rulesList.loc[rowCount] = [ ', '.join(affinity.items_base) ,\
                                    affinity.items_add, \
                                    len(affinity.items_base) ,\
                                    affinity.confidence,\
                                    affinity.lift]
        rowCount +=1


## Usando as regras

In [ ]:
rulesList.head()

,LHS,RHS,COUNT,CONFIDENCE,LIFT
0,,(AGE_GROUP=20 - 30),0,0.34,1.0
1,,(AGE_GROUP=30 - 50),0,0.32,1.0
2,,(AGE_GROUP=50PLUS ),0,0.16,1.0
3,,(AGE_GROUP=< 20),0,0.18,1.0
4,,(EMP_STATUS=EMPLOYED),0,0.54,1.0


Filtrando as regras quando a contagem de elementos for 1 e a confiança for > 70%

In [ ]:
rulesList[(rulesList.COUNT <= 1) & (rulesList.CONFIDENCE > 0.7)].head(5)

,LHS,RHS,COUNT,CONFIDENCE,LIFT
38,LIFETIME=3M to 1Y,(AGE_GROUP=20 - 30),1,1.000000,2.941176
70,AGE_GROUP=20 - 30,(TYPE=CANCEL),1,0.941176,1.568627
79,AGE_GROUP=30 - 50,(LIFETIME=1Y - 2Y),1,1.000000,3.125000
80,LIFETIME=1Y - 2Y,(AGE_GROUP=30 - 50),1,1.000000,3.125000
83,MARITAL_STATUS=MARRIED,(AGE_GROUP=30 - 50),1,0.833333,2.604167


Observando as regras, podemos ver facilmente alguns padrões. Clientes que deixaram a empresa entre três meses e um ano estão sempre na faixa etária de 20 a 30 anos. Da mesma forma, clientes na faixa etária de 20 a 30 anos sempre cancelaram o serviço. Esses são fatos interessantes que podem ser analisados mais a fundo pela empresa.